In [7]:
import numpy as np
import math
from simplified_connectx import *

ModuleNotFoundError: No module named 'gym'

In [4]:
temp = SimplifiedConnectX()

In [ ]:
def monte_carlo_tree_search(root):
     
    while resources_left(time, computational power):
        leaf = traverse(root)
        simulation_result = rollout(leaf)
        backpropagate(leaf, simulation_result)
         
    return best_child(root)
 
    # function for node traversal
    def traverse(node):
        while fully_expanded(node):
            node = best_uct(node)

        # in case no children are present / node is terminal
        return pick_unvisited(node.children) or node

    # function for the result of the simulation
    def rollout(node):
        while non_terminal(node):
            node = rollout_policy(node)
        return result(node)

    # function for randomly selecting a child node
    def rollout_policy(node):
        return pick_random(node.children)

    # function for backpropagation
    def backpropagate(node, result):
        if is_root(node) return
        node.stats = update_stats(node, result)
        backpropagate(node.parent)

    # function for selecting the best child
    # node with highest number of visits
    def best_child(node):
        pick child with highest number of visits

In [6]:
# Chat GPT code
import random

class MCTS:
    def __init__(self, game, iterations):
        self.game = game
        self.iterations = iterations

    def search(self, state):
        root = Node(state)

        for i in range(self.iterations):
            node = root
            # Select a leaf node to expand
            while not node.is_leaf():
                node = self.select(node)

            # Expand the leaf node and simulate a game from it
            if not node.is_terminal():
                node = self.expand(node)
                reward = self.simulate(node.state)
            else:
                reward = node.state.utility()

            # Backpropagate the reward up the tree
            while node is not None:
                node.update(reward)
                node = node.parent

        # Return the child with the highest number of visits
        return self.select_best_child(root)

    def select(self, node):
        # Select the child with the highest UCB1 score
        return max(node.children, key=lambda c: c.ucb1())

    def expand(self, node):
        # Add a random child to the node
        child_state = random.choice(node.state.children())
        child = Node(child_state, node)
        node.add_child(child)
        return child

    def simulate(self, state):
        # Play a random game from the current state and return the reward
        while not state.is_terminal():
            state = random.choice(state.children())
        return state.utility()

    def select_best_child(self, node):
        # Return the child with the highest number of visits
        return max(node.children, key=lambda c: c.visits)

class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.total_reward = 0

    def is_leaf(self):
        return len(self.children) == 0

    def is_terminal(self):
        return self.state.is_terminal()

    def add_child(self, child):
        self.children.append(child)

    def update(self, reward):
        self.visits += 1
        self.total_reward += reward

    def ucb1(self):
        # Calculate the UCB1 score for this node
        if self.visits == 0:
            return float('inf')

        exploration_term = math.sqrt(2 * math.log(self.parent.visits) / self.visits)
        return self.total_reward / self.visits + exploration_term

##### In MCTS we store 1 tree

In [1]:
class my_mcts():
    '''
    Implementation of MCTS algorithm based on the lecture 9
    from David Silver's course on youtube.
    
    Before leaf node is found, we run UCB, and after that we run
    the current policy.
    
    Leaf node is defined as a node which has not been seen by the UCB algorithm,
    Each time a node is found which is not seen, it gets added into the UCB
    tree (which is S_counts).
    '''
    def __init__(self,cur_state,cur_policy,args):
        '''
        state - It is the root state (in this case the game (which is a wrapper over
        the simplifiedconnectx gym environment))
        cur_policy - This is the policy which is run when the leaf node is found
        args - this has other arguments
        
        '''
        self.game = cur_state
        #This is the dictionary which keeps track of the score for all states and actions
        self.SA_score = {}
        #This stores the state action pair counts
        self.SA_counts = {}
        # This is the dictionary which counts the number of times a state is visited
        self.S_counts = {}
        # This stores the total number of times a simulation has been run
        self.total_counts = {}
        # This is the exploration constant, which governs how much we should explore
        # Higher c means higher exploration
        self.c = self.args["c"]
        
    def getActionProb(game,opt_player):
        '''
        This function runs all the simulations of the MCTS starting
        from the root node and then returns the final prob distribution
        
        arguments - 
        game: this is the current state of the board
        opt_player: this is the player to optimize for
        '''
        for i in range(self.args["num_mcts_sims"]):
            self.search(game.create_copy(),opt_player)
        
        s = game.stringRepresentation()
        # Lets get the scores of children of the current state s 
        prob = [self.SA_score[(s,a)] if (s,a) in self.SA_score else 0\
                for a in range(game.getActionSize())]
        # Now we need to normalize these scores
        total = sum(prob)
        
        if total == 0:
            # should never reach here
            print("There divide by zero in getActionProb")
        # Normalizing the probabilities
        prob = [x/total for x in prob] 
        
        print("The probabilities for state S: ",s," are :", prob)
        
        return prob
        
    def search(game,opt_player):
        '''
        This actually runs 1 episode from the total number of simulations
        
        If we are not at the leaf node, we should use UCB to pick which node to expand,
        If we are at the leaf node, we should use the current policy to expand the tree
        
        '''
        s = game.stringRepresentation()
        
        
        #check if the game has ended
        game_result = game.getGameEnded(opt_player)
        if game_result != 0:
            # The game has ended
            return game_result
        #check for leaf node
        if s not in self.S_counts:
            canonicalBoard = game.getCanonicalForm()
            q_score,v_score = self.nnet.predict(canonicalBoard)
            valids = game.getValidMoves()
            q_score = q_score * valids  # masking invalid moves
            sum_Ps_s = np.sum(q_score)
            if sum_Ps_s > 0:
                # q_score represents the prob distribution of our NN
                q_score /= sum_Ps_s  # renormalize
            else:
                # if all valid moves were masked make all valid moves equally probable

                # NB! All valid moves may be masked if either your NNet architecture is insufficient or you've get overfitting or something else.
                # If you have got dozens or hundreds of these messages you should pay attention to your NNet and/or training process.   
                log.error("All valid moves were masked, doing a workaround.")
                q_score = q_score + valids
                q_score /= np.sum(q_score)
            self.S_counts += 1
            # Lets choose a move from the prob distribution of this NN
            np.choose
        else:
            self.S_counts += 1
            # We have to apply UCB as this has been visited before
            best_action = None
            best_action_score = -float("inf")
            valids = game.getValidMoves()
            for action,move in enumerate(valids):
                # action is valid
                if move != 0:
                    #check if we have visited this action
                    if (s,action) in self.SA_score:
                        UCB_score = self.SA_score[(s,action)] + c*(math.sqrt(self.total_counts))/(self.SA_counts[(s,action)])
                    else:
                        # This is max because self.SA_counts[(s,action)] = 0 which is the denominator
                        UCB_score = float("inf")
                    if UCB_score > best_action_score:
                        best_action_score = UCB_score
                        best_action = action
            
            # Now that we have the best action, lets move it and continue
            nest_s, reward, done, info = game.getNextState(best_action)
            result = self.search(game,opt_player)
            
            # updating the score for the current state
            if (s, a) in self.SA_score:
                self.SA_score[(s, a)] = (self.SA_counts[(s, a)] * self.SA_score[(s, a)] + result) / (self.SA_counts[(s, a)] + 1)
                self.SA_counts[(s, a)] += 1
            else:
                self.SA_score[(s, a)] = result
                self.SA_counts[(s, a)] = 1
            #passing the result to the parent
            return result
        
        
        
        

SyntaxError: invalid syntax (516595999.py, line 31)

1

##### MCTS Variant, instead of using UCB_score to be infinite and forcing the 
##### Algorithm to explore the whole space, we can "guide" it with NN predictions
##### example p*(math.sqrt(self.total_counts))/(self.SA_counts[(s,action)])
##### Where p is the probability of picking that action according to the NN.


In [3]:
string = "This is some {x}"
string = string.format(x="string")
string

'This is some string'